In [ ]:
import rospy
import rosnode
import numpy as np
from franka_setEE import SetEEFrameService, SetLoadService
from franka_controllers import SwitchControllersSrv
from franka_msgs.msg import FrankaState
from franka_cartesian_impedance_param import SetCartesianImpedanceParams

print("Initializing...")
rospy.init_node("franka_test", anonymous=True)

set_ee_service = SetEEFrameService()
set_load_service = SetLoadService()
switch_controllers = SwitchControllersSrv()
set_impedance = SetCartesianImpedanceParams()
print("Initialized.")

set_impedance.set_impedance(trans=400.0, rot=20.0)

In [ ]:
state = rospy.wait_for_message("/franka_state_controller/franka_states", FrankaState, timeout=1)
print(state.NE_T_EE)
switch_controllers.send_request(start_controllers=[""],
                                stop_controllers=["cartesian_impedance_example_controller"])
rospy.sleep(1)
response = set_load_service.send_request(mass=0.76,
                              F_x_center_load=[0.2, 0.1, 0.15],
                              load_inertia=[0.001333, 0.0,     0.0,
                                            0.0,      0.00347, 0.0,
                                            0.0,      0.0,     0.00453])
T = np.array([[1.0, 0.0, 0.0, 0.2],
              [0.0, 1.0, 0.0, 0.2],
              [0.0, 0.0, 1.0, 0.2],
              [0.0, 0.0, 0.0, 1.0]])
new_mtx = np.reshape(T, (16), order = 'F')
response = set_ee_service.send_request(NE_T_EE=new_mtx)

switch_controllers.send_request(start_controllers=["cartesian_impedance_example_controller"],
                                stop_controllers=[""])
rospy.sleep(1)
state = rospy.wait_for_message("/franka_state_controller/franka_states", FrankaState, timeout=1)
print(state.NE_T_EE)